In [33]:
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/germa/thesis/bachelor_project2023/src/utils')
from chunker import Chunker
import torch.nn as nn
import torch
from torch import tensor


In [25]:
train_df = pd.read_csv('../../data/train/train.csv')
train_short_df = train_df.head(100)
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
chunker = Chunker(tokenizer, 512)

In [28]:
train_short_df["text1_chunks"]=train_short_df["text1"].apply(lambda x: chunker.chunk(x))
train_short_df["text2_chunks"]=train_short_df["text2"].apply(lambda x: chunker.chunk(x))


/var/folders/c6/k78nzngn7nbdd0786bxl3s0c0000gn/T/ipykernel_1628/4139101259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_short_df["text1_chunks"]=train_short_df["text1"].apply(lambda x: chunker.chunk(x))
/var/folders/c6/k78nzngn7nbdd0786bxl3s0c0000gn/T/ipykernel_1628/4139101259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_short_df["text2_chunks"]=train_short_df["text2"].apply(lambda x: chunker.chunk(x))


In [30]:
#count per row how many chunks there are for each text
train_short_df["text1_chunks_count"]=train_short_df["text1_chunks"].apply(lambda x: len(x)) 
train_short_df["text2_chunks_count"]=train_short_df["text2_chunks"].apply(lambda x: len(x))


/var/folders/c6/k78nzngn7nbdd0786bxl3s0c0000gn/T/ipykernel_1628/1495922066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_short_df["text1_chunks_count"]=train_short_df["text1_chunks"].apply(lambda x: len(x))
/var/folders/c6/k78nzngn7nbdd0786bxl3s0c0000gn/T/ipykernel_1628/1495922066.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_short_df["text2_chunks_count"]=train_short_df["text2_chunks"].apply(lambda x: len(x))


In [35]:
class LSTMOnXLMRoberta(nn.Module):
    def __init__(self, xlmroberta_model, lstm_hidden_size, num_lstm_layers):
        super(LSTMOnXLMRoberta, self).__init__()
        self.xlmroberta_model = XLMRobertaModel.from_pretrained(xlmroberta_model)
        self.lstm = nn.LSTM(input_size=xlmroberta_model.config.hidden_size,
                            hidden_size=lstm_hidden_size,
                            num_layers=num_lstm_layers,
                            batch_first=True,
                            bidirectional=False)
        self.fc = nn.Linear(lstm_hidden_size * 2, 1)
    
    def _manage_device(self) -> None:
        """
            Manage the device to run the model on
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.xlmroberta_model.to(self.device)

    def forward(self, input_ids_list):
        pooled_outputs = []
        for input_ids in input_ids_list:
            outputs = self.xlmroberta_model(input_ids=torch.tensor(input_ids).to(outputs.last_hidden_state.device))
            pooled_output = torch.mean(outputs.last_hidden_state, dim=1)  # Average pooling
            pooled_outputs.append([pooled_output])
    
        # Stack pooled outputs into a tensor
        pooled_outputs = torch.stack(pooled_outputs, dim=0)

        lstm_out, _ = self.lstm(pooled_outputs.unsqueeze(1))
        logits = self.fc(lstm_out.squeeze(1))
        return logits

    def train(self, train_loader, val_loader, device, model, optimizer, criterion, scheduler, num_epochs=5):
        for epoch in range(num_epochs):
            for idx, (ids, att, val) in enumerate(train_loader):
                ids, att, val = ids.to(self.device), att.to(self.device), val.to(self.device)
                logits = self.forward(ids)

        pass

        